## Data Ingestion

In [7]:
# Import libraries and modules
import pandas as pd
import minsearch
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")
from tqdm.auto import tqdm
from openai import OpenAI

C:\Users\user\.virtualenvs\Project-v4EWajkE\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
df=pd.read_csv('../data/clean_data/Final_QandA_split_conversations_with_keywords.csv')
documents = df.to_dict(orient='records')

In [16]:
print(len(documents))
documents[0]


49006


{'Chunked_Content': "when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren't at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks",
 'number of sentences': 2,
 'number of words': 69,
 'Keywords': "['pass', 'meal', 'home', 'nausea', 'honey', 'week', 'hunger', 'second', 'previous', 'btw', '20h+', 'kid', 'good', 'typical', 'idea', 'enhance', 'zero', '4', 'today', 'eat', 'yolk', 'egg', 'yesterday', 'wuth', 'hour', 'end']"}

In [17]:
index = minsearch.Index(
    text_fields=['Chunked_Content'],
    keyword_fields=['Keywords']
)

In [18]:
index.fit(documents)

## RAG Flow

In [20]:
client = OpenAI()

In [22]:
query = 'I want to lose belly fat. What shall I eat?'

In [23]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [25]:
search(query)

[{'Chunked_Content': 'And lose Belly fat by doing nothing?. Body positivity',
  'number of sentences': 2,
  'number of words': 9,
  'Keywords': "['lose', 'fat', 'Belly', 'body', 'positivity']"},
 {'Chunked_Content': 'Why is your belly so big?',
  'number of sentences': 1,
  'number of words': 6,
  'Keywords': "['belly', 'big']"},
 {'Chunked_Content': 'what if i want to stay lean and keep it like that?. is that "healthy"?. Fat is beautifull🥹 no, i just have lube 1x a day 25% BF but look like 7% is that a genetic factor?what about the fat you\'d gain around your belly and chubby face?im okay with not having a six pack, i just dont wanna have lose rolls of fat on my sides and belly',
  'number of sentences': 3,
  'number of words': 69,
  'Keywords': "['lose', 'Fat', 'beautifull🥹', '7', 'not', 'lean', '1x', 'healthy', 'roll', 'lube', 'look', '25', 'okay', 'chubby', 'belly', 'stay', 'day', 'want', 'genetic', 'face?im', 'pack', 'have', 'like', 'gain', 'fat', 'BF', 'factor?what', 'side', 'wan

In [28]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": query}]
)

response.choices[0].message.content
print(_)

Losing belly fat often involves a combination of a healthy diet, regular exercise, and lifestyle changes. Here’s a general guide on what to eat to help reduce belly fat:

### **1. Incorporate More Whole Foods:**
- **Vegetables:** Leafy greens, broccoli, Brussels sprouts, bell peppers, and other non-starchy vegetables provide fiber and essential nutrients.
- **Fruits:** Berries, apples, oranges, and other fruits are high in fiber and antioxidants.
- **Whole Grains:** Brown rice, quinoa, oats, and whole wheat products can help keep you full, thanks to their fiber content.

### **2. High-Quality Protein:**
- **Lean Meat:** Chicken breast, fish, turkey, and lean cuts of beef.
- **Plant-Based Proteins:** Lentils, chickpeas, beans, tofu, and tempeh.
- **Eggs:** A good source of protein and healthy fats.

### **3. Healthy Fats:**
- **Avocados:** Rich in monounsaturated fats.
- **Nuts and Seeds:** Almonds, chia seeds, flaxseeds, and walnuts.
- **Olive Oil:** Use it as your primary oil for cook

In [29]:
prompt_template = """
You're a health adviser. Answer the QUESTION based on the CONTEXT from our primal diet database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
Chunked_Content: {Chunked_Content}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [48]:
def llm(prompt, model='gpt-4o'):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [69]:
query='How to lower blood pressure?'
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, model=model)
    return answer



In [70]:
print(rag(query))

Based on the information from the primal diet context provided by Aajonus, here is the advice on how to lower blood pressure:

1. **Identify the Cause**: Acknowledge that high blood pressure may occur for valid reasons such as being overweight, having arterial congestion, or being exposed to certain toxins. It's crucial to address these underlying causes rather than just focusing on reducing the pressure itself.

2. **Dietary Adjustments**:
   - **Grapefruit**: Consuming half a grapefruit daily for 10 days can help lower blood pressure if it is due to a lack of specific nutrients required for this function.
   - **Cucumbers**: Eating raw cucumbers or drinking cucumber juice can soothe stretched arteries, veins, and nerves.
   - **Garlic**: Fresh raw garlic can help stabilize blood pressure, whether it is too high or too low.

3. **Note on Medications**: Be cautious with blood pressure medications as they may slow down blood flow through congested arteries, potentially causing clots.

4